欠損値穴埋め中央値
主成分分析
keras(活性化関数:シグモイド(ロジスティック)関数，最適化アルゴリズム:アダム，損失関数:バイナリクロスエントロピー2クラス分類用,評価関数:accuracy,kernel initializer:重みの初期化は一様)


In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
from sklearn.feature_selection import VarianceThreshold
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
import numpy as np
import random
import keras.backend as K
import tensorflow as tf

np.random.seed(seed=0)

# os.environ['PYTHONHASHSEED'] = '0'
# random.seed(0)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [3]:
df_train = pd.read_csv('secom_train_data.tsv', delimiter='\t')
df_test = pd.read_csv('secom_production_data_.csv')

In [4]:
df_train = pd.DataFrame(df_train)
df_train.head()

,failure,timestamp,s1,s2,s3,s4,s5,s6,s7,s8,...,s581,s582,s583,s584,s585,s586,s587,s588,s589,s590
0,-1,2008/07/19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,...,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN
1,-1,2008/07/19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,...,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,1,2008/07/19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,...,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
3,-1,2008/07/19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,...,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
4,-1,2008/07/19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,...,NaN,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Columns: 592 entries, failure to s590
dtypes: float64(586), int64(5), object(1)
memory usage: 5.4+ MB


In [6]:
df_train = df_train.fillna(df_train.median())
df_test = df_test.fillna(df_test.median())

C:\Users\tamaru\Anaconda3\envs\tensorflow16\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [7]:
df_train = df_train.drop(columns=['timestamp'])
df_test = df_test.drop(columns=['timestamp']) 
df_train_feature = df_train.drop(columns=['failure'])
df_test_feature = df_test.drop(columns=['failure'])
df_train_feature.head()

,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,...,s581,s582,s583,s584,s585,s586,s587,s588,s589,s590
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,0.0047,70.3693,0.5005,0.0118,0.0035,2.3630,0.0210,0.0149,0.0046,70.1798
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,0.0047,70.3693,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432


In [8]:
pca = PCA()
pca.fit(df_train_feature)
pca.fit(df_test_feature)
train_feature_ = pca.transform(df_train_feature)
test_feature_ = pca.transform(df_test_feature)
train_feature_ = pd.DataFrame(train_feature_)
test_feature_ = pd.DataFrame(test_feature_)
train_feature_.head()

,0,1,2,3,4,5,6,7,8,9,...,357,358,359,360,361,362,363,364,365,366
0,-6401.748718,-2308.485867,907.206238,-300.900536,-113.336316,49.145256,-103.023495,319.890144,58.399207,933.174369,...,0.019330,-0.006703,0.028865,0.060293,0.028908,-0.007811,-0.009631,0.026500,-0.026318,0.016138
1,-3243.717943,-3031.036529,-1568.921840,261.350141,-181.823036,-71.890080,14.531545,-405.790946,11.054553,886.378402,...,0.016799,0.000799,-0.010732,0.011914,-0.008176,-0.018929,-0.005005,0.022470,0.003710,0.002466
2,-4503.841497,-2370.442696,-1755.915438,-327.852291,-168.152489,167.610323,-66.885451,-125.081392,-172.257138,-435.169008,...,0.405440,1.542683,0.882581,1.279738,-1.341129,-0.353529,0.529652,0.388826,-0.274660,0.313204
3,-3221.914340,378.511103,-982.107699,-666.514935,-54.902562,-529.866518,-92.548382,142.122393,-160.909272,323.739736,...,0.440599,1.449877,0.750485,1.322071,-1.339448,-0.206053,0.270655,0.337142,-0.405474,0.274092
4,-5159.248161,-1584.091547,427.440357,-256.717194,-127.036796,-175.959457,-32.556906,-60.135210,-93.844122,183.611134,...,0.074466,-0.050720,0.034035,0.053439,0.038134,0.043471,-0.072145,0.040836,-0.121493,-0.002497


In [9]:
# estimator = LogisticRegression(random_state=0)
# print(1)
# selector = RFE(estimator, 1, step=1, verbose=1)
# print(2)
# selector = estimator.fit(train_feature_, df_train['failure'])
# print(3)
# selector.support_ 

In [10]:
# for i in range(len(selector)):
#     if(selector[i] == False):
#         train_feature_.drop(i,axis=1)
# train_feature_.head()

In [11]:
model = Sequential()
model.add(Dense(1, input_dim=len(train_feature_.columns), kernel_initializer='uniform'))
model.add(Activation('sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

W0902 16:58:36.504893 10976 deprecation_wrapper.py:119] From C:\Users\tamaru\Anaconda3\envs\tensorflow16\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0902 16:58:36.512871 10976 deprecation_wrapper.py:119] From C:\Users\tamaru\Anaconda3\envs\tensorflow16\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0902 16:58:36.519868 10976 deprecation_wrapper.py:119] From C:\Users\tamaru\Anaconda3\envs\tensorflow16\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0902 16:58:36.555757 10976 deprecation_wrapper.py:119] From C:\Users\tamaru\Anaconda3\envs\tensorflow16\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W090

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
hist = model.fit(train_feature_, df_train['failure'],
                 epochs=100, verbose=1,
                 validation_split=0.1,
                 callbacks=[early_stopping])

W0902 16:58:36.919815 10976 deprecation_wrapper.py:119] From C:\Users\tamaru\Anaconda3\envs\tensorflow16\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 1080 samples, validate on 120 samples
Epoch 1/100
1080/1080 [==============================] - 1s 887us/step - loss: -4.5427 - acc: 0.0130 - val_loss: 0.5191 - val_acc: 0.0167
Epoch 2/100
1080/1080 [==============================] - 0s 81us/step - loss: -5.2754 - acc: 0.0111 - val_loss: 0.3552 - val_acc: 0.0167
Epoch 3/100
1080/1080 [==============================] - 0s 71us/step - loss: -6.1014 - acc: 0.0083 - val_loss: -0.5958 - val_acc: 0.0083
Epoch 4/100
1080/1080 [==============================] - 0s 64us/step - loss: -6.8923 - acc: 0.0074 - val_loss: -1.3998 - val_acc: 0.0083
Epoch 5/100
1080/1080 [==============================] - 0s 72us/step - loss: -7.0084 - acc: 0.0093 - val_loss: -1.5355 - val_acc: 0.0083
Epoch 6/100
1080/1080 [==============================] - 0s 113us/step - loss: -7.3624 - acc: 0.0056 - val_loss: 0.4678 - val_acc: 0.0083
Epoch 7/100
1080/1080 [==============================] - 0s 91us/step - loss: -7.6013 - acc: 0.0046 - val_loss: -1.3731 - val_

In [13]:
predict = model.predict(test_feature_)
print(predict)

[[1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [9.9576885e-01]
 [1.0000000e+00]
 [2.2351742e-06]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [2.3841858e-07]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+0

In [14]:
for i in range(len(predict)):
    if predict[i] < 0.5:
        predict[i] = -1
    else:
        predict[i] = 1
predict = pd.DataFrame(predict)
predict.columns = ['answer']

In [15]:
predict.to_csv("predict_median.csv", header=True, index=False)